<a href="https://colab.research.google.com/github/abhinavmarkanda/UCS547-Accelerated-Data-Science/blob/main/Assignment2(UCS547).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
##Q1
##1. ! Run System(Shell Commands)
!pwd
!ls
!pip install numpy




/content
sample_data


In [5]:
##2 % Line Magic Commands
%time sum(range(1000000))


CPU times: user 35.3 ms, sys: 880 µs, total: 36.2 ms
Wall time: 36.1 ms


499999500000

In [6]:
##2 % Line Magic Commands
%lsmagic



Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%late

In [8]:
##3 %% Cell Magic Commands
%%time
total = 0
for i in range(1000000):
    total += i

CPU times: user 112 ms, sys: 910 µs, total: 113 ms
Wall time: 113 ms


In [9]:
##3 %% Cell Magic Commands
%%writefile test.py
print("Hello Abhinav")



Writing test.py


In [10]:
##3 %% Cell Magic Commands
%%bash
echo "Hello"
pwd


Hello
/content


In [13]:
##Q2
##1 Check If GPU Is Enabled
!nvidia-smi

##2 Check GPU Name Only
!nvidia-smi --query-gpu=name --format=csv

##3 Check GPU Memory Usage
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

##4 Monitor GPU Continuously
!nvidia-smi -l 1

##5 Check GPU Utilization
!nvidia-smi --query-gpu=utilization.gpu --format=csv

##6 Check Temperature
!nvidia-smi --query-gpu=temperature.gpu --format=csv

##7 Check Running Processes
!nvidia-smi --query-compute-apps=pid,process_name,used_memory --format=csv!nvidia-smi -q

##8 Detailed Information
!nvidia-smi -q






Wed Feb 18 04:41:07 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             14W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [23]:
##Q3
%%writefile cuda_debug.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void add(int *a, int *b, int *c, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int N = 10;
    int a[N], b[N], c[N];

    for(int i=0;i<N;i++){
        a[i] = i;
        b[i] = i;
    }

    int *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, N*sizeof(int));
    cudaMalloc((void**)&d_b, N*sizeof(int));
    cudaMalloc((void**)&d_c, N*sizeof(int));

    cudaMemcpy(d_a, a, N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, N*sizeof(int), cudaMemcpyHostToDevice);

    add<<<1, N>>>(d_a, d_b, d_c, N);
    cudaDeviceSynchronize();

    cudaMemcpy(c, d_c, N*sizeof(int), cudaMemcpyDeviceToHost);

    for(int i=0;i<N;i++)
        printf("%d ", c[i]);

    printf("\n");

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


Writing cuda_debug.cu


In [24]:
!nvcc cuda_debug.cu -o cuda_debug


nvcc warning : Support for offline compilation for architectures prior to '<compute/sm/lto>_75' will be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [25]:
!./cuda_debug


0 2 4 6 8 10 12 14 16 18 


In [ ]:
##Q4
%%writefile hello.cu
#include <stdio.h>
#include <cuda_runtime.h>

///////////////////////////////////////////////////////////
// DEVICE CODE (GPU Kernel)
///////////////////////////////////////////////////////////

__global__ void helloKernel()
{
    int global_thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Hello from GPU thread %d\n", global_thread_id);
}

///////////////////////////////////////////////////////////
// HOST CODE (CPU)
///////////////////////////////////////////////////////////

int main()
{
    printf("Hello from CPU (Host)\n");

    // Launch kernel: 1 block, 8 threads
    helloKernel<<<1, 8>>>();

    // Wait for GPU to finish
    cudaDeviceSynchronize();

    printf("Back to CPU after kernel execution\n");

    return 0;
}


Writing hello.cu


In [ ]:
!nvcc hello.cu -o hello


nvcc warning : Support for offline compilation for architectures prior to '<compute/sm/lto>_75' will be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
!./hello


Hello from CPU (Host)
Hello from GPU thread 0
Hello from GPU thread 1
Hello from GPU thread 2
Hello from GPU thread 3
Hello from GPU thread 4
Hello from GPU thread 5
Hello from GPU thread 6
Hello from GPU thread 7
Back to CPU after kernel execution


In [ ]:
##Q5
%%writefile memory_demo.cu
#include <stdio.h>
#include <cuda_runtime.h>

///////////////////////////////////////////////////////////
// DEVICE CODE (Runs on GPU)
///////////////////////////////////////////////////////////

__global__ void printFromDevice(int *d_array)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < 5)
    {
        printf("GPU Thread %d reads value: %d\n", tid, d_array[tid]);

        // Modify value on GPU (for demonstration)
        d_array[tid] = d_array[tid] * 2;
    }
}

///////////////////////////////////////////////////////////
// HOST CODE (Runs on CPU)
///////////////////////////////////////////////////////////

int main()
{
    // a. Create integer array on Host (CPU)
    int h_array[5] = {10, 20, 30, 40, 50};
    int *d_array;

    printf("----- Host (CPU) Original Data -----\n");
    for(int i = 0; i < 5; i++)
        printf("h_array[%d] = %d\n", i, h_array[i]);

    // b. Allocate memory on Device (GPU)
    cudaMalloc((void**)&d_array, 5 * sizeof(int));

    // c. Copy data from Host to Device
    cudaMemcpy(d_array, h_array, 5 * sizeof(int), cudaMemcpyHostToDevice);

    // d. Launch kernel (1 block, 5 threads)
    printFromDevice<<<1, 5>>>(d_array);

    cudaDeviceSynchronize();

    // e. Copy modified data back to Host
    cudaMemcpy(h_array, d_array, 5 * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\n----- Host (CPU) After GPU Modification -----\n");
    for(int i = 0; i < 5; i++)
        printf("h_array[%d] = %d\n", i, h_array[i]);

    // Free device memory
    cudaFree(d_array);

    return 0;
}


Writing memory_demo.cu


In [ ]:
!nvcc memory_demo.cu -o memory_demo


nvcc warning : Support for offline compilation for architectures prior to '<compute/sm/lto>_75' will be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
!./memory_demo


----- Host (CPU) Original Data -----
h_array[0] = 10
h_array[1] = 20
h_array[2] = 30
h_array[3] = 40
h_array[4] = 50
GPU Thread 0 reads value: 10
GPU Thread 1 reads value: 20
GPU Thread 2 reads value: 30
GPU Thread 3 reads value: 40
GPU Thread 4 reads value: 50

----- Host (CPU) After GPU Modification -----
h_array[0] = 20
h_array[1] = 40
h_array[2] = 60
h_array[3] = 80
h_array[4] = 100


In [1]:
##Q6
import time
import numpy as np

N = 10_000_000   # 10 million elements

# -----------------------------
# 1️⃣ LIST
# -----------------------------
list1 = list(range(N))
list2 = list(range(N))

start = time.time()
result_list = [list1[i] + list2[i] for i in range(N)]
end = time.time()

print("List Time:", end - start)


# -----------------------------
# 2️⃣ TUPLE
# -----------------------------
tuple1 = tuple(range(N))
tuple2 = tuple(range(N))

start = time.time()
result_tuple = tuple(tuple1[i] + tuple2[i] for i in range(N))
end = time.time()

print("Tuple Time:", end - start)


# -----------------------------
# 3️⃣ NUMPY ARRAY
# -----------------------------
array1 = np.arange(N)
array2 = np.arange(N)

start = time.time()
result_array = array1 + array2
end = time.time()

print("NumPy Time:", end - start)


List Time: 0.9722373485565186
Tuple Time: 0.9884428977966309
NumPy Time: 0.030039548873901367
